<h1>Ensaio para Active Voxel</h1>

In [6]:
from vpython import *

ctr = vec( 150, 50, 150 )

# This is needed in Jupyter notebook and lab to make programs easily rerunnable
scene = canvas( center=ctr, background = color.gray( 0.8 ))
scene.autoscaling = False
scene.forward = vec( 0, 200, 0 )
scene.up = vec( 0, 0, 1 )
scene.camera.pos = vec( 150, -330, 150 )
scene.range = 50

<IPython.core.display.Javascript object>

In [13]:
# quads always start with 6 quads needed to form the seed
# <(( int e1, int e2 ) t1, ( int e3, int e4 ) t2 ), int >
quads = {
    # front
    (( 1, 2 ), ( 3, 4 )): 0,

    # top
    (( -3, 5 ), ( 6, 7 )): 1,

    # back
    (( -6, 8 ), ( 9, 10 )): 2,

    # bottom
    (( -9, 11 ), ( -1, 12 )): 3,

    # left
    (( -11, -8 ), ( -5, -2 )): 4,

    # right
    (( -12, -4 ), ( -7, -10 )): 5
}

print( quads )

{((1, 2), (3, 4)): 0, ((-3, 5), (6, 7)): 1, ((-6, 8), (9, 10)): 2, ((-9, 11), (-1, 12)): 3, ((-11, -8), (-5, -2)): 4, ((-12, -4), (-7, -10)): 5}


In [16]:
quads_rev = {}

# dict comprehension failed here
for quad in quads:
    quads_rev[ quads[ quad ]] = quad
    
print( quads_rev )

{0: ((1, 2), (3, 4)), 1: ((-3, 5), (6, 7)), 2: ((-6, 8), (9, 10)), 3: ((-9, 11), (-1, 12)), 4: ((-11, -8), (-5, -2)), 5: ((-12, -4), (-7, -10))}
